In [115]:
import numpy as np
import pandas as pd

In [116]:
df=pd.read_csv('AMZN_stock_data.csv')
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1997-05-15 00:00:00-04:00,0.121875,0.125000,0.096354,0.097917,1443120000,0.0,0.0
1,1997-05-16 00:00:00-04:00,0.098438,0.098958,0.085417,0.086458,294000000,0.0,0.0
2,1997-05-19 00:00:00-04:00,0.088021,0.088542,0.081250,0.085417,122136000,0.0,0.0
3,1997-05-20 00:00:00-04:00,0.086458,0.087500,0.081771,0.081771,109344000,0.0,0.0
4,1997-05-21 00:00:00-04:00,0.081771,0.082292,0.068750,0.071354,377064000,0.0,0.0
...,...,...,...,...,...,...,...,...
6982,2025-02-14 00:00:00-05:00,229.199997,229.889999,227.229996,228.679993,27031100,0.0,0.0
6983,2025-02-18 00:00:00-05:00,228.820007,229.300003,223.720001,226.649994,42975100,0.0,0.0
6984,2025-02-19 00:00:00-05:00,225.520004,226.830002,223.710007,226.630005,28566700,0.0,0.0
6985,2025-02-20 00:00:00-05:00,224.779999,225.130005,221.809998,222.880005,30001700,0.0,0.0


In [117]:
df['Date']= pd.to_datetime(df['Date'])

/var/folders/n_/1hzw47553v1bvg9z8kct6rd40000gn/T/ipykernel_40725/2691600196.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date']= pd.to_datetime(df['Date'])


In [118]:
df['Close'].describe()

count    6987.000000
mean       40.685061
std        58.125169
min         0.069792
25%         2.110000
50%         9.021000
75%        73.006500
max       242.059998
Name: Close, dtype: float64

In [119]:
df['High'].describe()

count    6987.000000
mean       41.165619
std        58.787641
min         0.072396
25%         2.148250
50%         9.158500
75%        73.643497
max       242.520004
Name: High, dtype: float64

In [120]:
df['Close']=df['Close'].apply(lambda x: 1 if x>40.685061 else 0)
df['High']=df['High'].apply(lambda x: 1 if x>41.165619 else 0)


In [121]:
final=df[['Date','High','Close']]

In [122]:
final.set_index('Date')

,High,Close
Date,,
1997-05-15 00:00:00-04:00,0,0
1997-05-16 00:00:00-04:00,0,0
1997-05-19 00:00:00-04:00,0,0
1997-05-20 00:00:00-04:00,0,0
1997-05-21 00:00:00-04:00,0,0
...,...,...
2025-02-14 00:00:00-05:00,1,1
2025-02-18 00:00:00-05:00,1,1
2025-02-19 00:00:00-05:00,1,1


In [123]:
from sklearn.model_selection import train_test_split
train_size = int(0.8 * len(final))
train_df = final.iloc[:train_size]
test_df = final.iloc[train_size:]

In [124]:
# Step 1: Build state and observation label lists
train_states = sorted(train_df['High'].unique()) 
train_observations = sorted(train_df['Close'].unique())
test_states = sorted(test_df['High'].unique()) 
test_observations = sorted(test_df['Close'].unique())

In [125]:
def calculate_emission_matrix(observations_seq, states_seq, obs_symbols=None, state_symbols=None, smoothing=1):
    if obs_symbols is None:
        obs_symbols = sorted(set(observations_seq))
    if state_symbols is None:
        state_symbols = sorted(set(states_seq))
    obs_idx = {obs: i for i, obs in enumerate(obs_symbols)}
    state_idx = {state: i for i, state in enumerate(state_symbols)}

    num_states = len(state_symbols)
    num_obs = len(obs_symbols)

    # Start with smoothing counts instead of zeros
    emission_matrix = np.ones((num_states, num_obs)) * smoothing

    for s, o in zip(states_seq, observations_seq):
        i = state_idx[s]
        j = obs_idx[o]
        emission_matrix[i, j] += 1

    # Normalize rows
    row_sums = emission_matrix.sum(axis=1, keepdims=True)
    emission_matrix = emission_matrix / row_sums

    return emission_matrix, state_symbols, obs_symbols

In [126]:
def calculate_transition_matrix(states_sequence, all_states=None, smoothing=0):
    if all_states is None:
        states = sorted(set(states_sequence))
    else:
        states = list(all_states)
    state_idx = {state:i for i, state in enumerate(states)}
    num_states = len(states)
    counts = np.zeros((num_states, num_states))
    for t in range(1, len(states_sequence)):
        i = state_idx[states_sequence[t-1]]
        j = state_idx[states_sequence[t]]
        counts[i, j] += 1
    if smoothing > 0:
        counts += smoothing
    row_sums = counts.sum(axis=1, keepdims=True)
    transition_matrix = np.divide(counts, row_sums, where=row_sums!=0)
    return transition_matrix, states

In [127]:
class HiddenMarkovModel:
    """A Hidden markov model which takes Transition matrix and Emmission matrix as inputs"""

    def __init__(self, transition_matrix, emission_matrix, prior=None):
      self.transition_matrix = transition_matrix
      self.emission_matrix = emission_matrix
      self.prior = prior
      return None

    def emission_dist(self, ev):
      return self.emission_matrix[:, ev]

In [128]:
def forward_step(Hmm,trans, fv, ev):
  emission_dist = Hmm.emission_dist(ev)
  forward_inf = fv
  transition_matrix = trans

  nonNormSolution = emission_dist * np.array([[1,0],[0,1]]) @transition_matrix @ forward_inf
  alpha = nonNormSolution / np.sum(nonNormSolution)


  return alpha

In [129]:
def forward_algorithm(Hmm,observations):
    T = len(observations)
    N = len(Hmm.prior)

    alpha = np.zeros((T,N))

    alpha[0] = Hmm.prior * Hmm.emission_dist(observations[0])
    alpha[0] = alpha[0] / np.sum(alpha[0])

    for t in range(1,T):
        alpha[t] = forward_step(Hmm,Hmm.transition_matrix, alpha[t-1],observations[t])
    return alpha

In [130]:
# ---- Backward algorithm ----
def backward_algorithm(obs, states, start_prob, trans_prob, emit_prob):
    N = len(states)
    T = len(obs)
    backward = np.zeros((T, N))
    backward[T-1, :] = 1
    for t in range(T-2, -1, -1):
        for i in range(N):
            backward[t, i] = np.sum(trans_prob[i, :] * emit_prob[:, obs[t+1]] * backward[t+1, :])
    prob = np.sum(start_prob * emit_prob[:, obs[0]] * backward[0, :])
    return backward, prob


In [131]:
def viterbi_algorithm(obs, states, start_prob, trans_prob, emit_prob):
    """
    obs: list or array of observations (as integer indices)
    states: list of state names or values
    start_prob: numpy array of shape (N,)
    trans_prob: numpy array of shape (N,N)
    emit_prob: numpy array of shape (N,M)

    Returns:
        best_path: list of most likely states (by label, not index)
        delta: np.ndarray of shape (T, N), max probability at each state, each t
        psi: np.ndarray of shape (T, N), backpointer for optimal path
    """
    N = len(states)
    T = len(obs)

    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)

    # Initialization
    delta[0, :] = start_prob * emit_prob[:, obs[0]]
    psi[0, :] = 0

    # Recursion
    for t in range(1, T):
        for j in range(N):
            # For each state j, find best prior state i
            seq_probs = delta[t-1, :] * trans_prob[:, j]
            psi[t, j] = np.argmax(seq_probs)
            delta[t, j] = np.max(seq_probs) * emit_prob[j, obs[t]]

    # Termination
    best_last_state = np.argmax(delta[T-1, :])
    best_path_idx = np.zeros(T, dtype=int)
    best_path_idx[T-1] = best_last_state

    # Path backtracking
    for t in range(T-2, -1, -1):
        best_path_idx[t] = psi[t+1, best_path_idx[t+1]]

    # Convert indices to state labels
    best_path = [states[i] for i in best_path_idx]

    return best_path, delta, psi


In [132]:
emit_prob, state_labels, obs_labels = calculate_emission_matrix(
    train_df['Close'].values, train_df['High'].values, train_observations, train_states
)
emit_prob

array([[0.99838188, 0.00161812],
       [0.00154083, 0.99845917]])

In [133]:
trans_prob, state_labels = calculate_transition_matrix(train_df['High'].values, train_states)
trans_prob

array([[9.99190611e-01, 8.09388911e-04],
       [4.64396285e-03, 9.95356037e-01]])

In [134]:
obs=list(train_df['Close'])
start_prob = np.ones(len(train_states)) / len(train_states)  # or a real prior if available
Hmm = HiddenMarkovModel(trans_prob, emit_prob, prior=start_prob)
forward_algorithm(Hmm,obs)

array([[9.98459049e-01, 1.54095115e-03],
       [9.99990454e-01, 9.54559145e-06],
       [9.99992812e-01, 7.18759375e-06],
       ...,
       [1.31997756e-06, 9.99998680e-01],
       [1.31997756e-06, 9.99998680e-01],
       [1.31997756e-06, 9.99998680e-01]])

In [135]:
N = len(train_states)
start_prob = np.ones(N) / N 

In [136]:
backward_matrix, sequence_prob = backward_algorithm(
    obs, train_states, start_prob, trans_prob, emit_prob
)

backward_matrix

array([[4.95596377e-30, 2.30694220e-32],
       [4.96801716e-30, 2.31255291e-32],
       [4.98009986e-30, 2.31817726e-32],
       ...,
       [8.07076143e-04, 9.87690371e-01],
       [2.42495507e-03, 9.93829875e-01],
       [1.00000000e+00, 1.00000000e+00]])

In [137]:
backward_matrix, sequence_prob = backward_algorithm(
    obs, train_states, start_prob, trans_prob, emit_prob
)

print("Backward probability matrix shape:", backward_matrix.shape)
print("Probability of observation sequence under the HMM:", sequence_prob)

Backward probability matrix shape: (5589, 2)
Probability of observation sequence under the HMM: 2.473989980208364e-30


In [138]:
N = len(state_labels)
start_prob = np.ones(N) / N

best_path, delta, psi = viterbi_algorithm(
    obs=obs,
    states=state_labels,
    start_prob=start_prob,
    trans_prob=trans_prob,
    emit_prob=emit_prob
)

# Map 0 to "below median ", 1 to "above median"
labels = ["below median" if state == 0 else "above median" for state in best_path]


# If you want to add this to your DataFrame
final = train_df.copy()  # avoid modifying original
final['path'] = labels

print(final[['High', 'Close', 'path']].to_string(index=False))

 High  Close         path
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0 below median
    0      0

In [139]:
# Training results 
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# True hidden states (e.g., High column)
true_states = train_df['High'].values

# Predicted hidden states from Viterbi
predicted_states = best_path

# Basic metrics
accuracy = accuracy_score(true_states, predicted_states)
conf_matrix = confusion_matrix(true_states, predicted_states)
precision = precision_score(true_states, predicted_states, zero_division=0)
recall = recall_score(true_states, predicted_states, zero_division=0)
f1 = f1_score(true_states, predicted_states, zero_division=0)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Accuracy: 0.9986
Confusion Matrix:
[[4934    8]
 [   0  647]]
Precision: 0.9878
Recall:    1.0000
F1 Score:  0.9939


In [140]:
emit_prob, state_labels, obs_labels = calculate_emission_matrix(
    test_df['Close'].values, test_df['High'].values, test_observations, test_states
)
emit_prob

array([[1.]])

In [141]:
trans_prob, state_labels = calculate_transition_matrix(test_df['High'].values, test_states)
trans_prob

array([[1.]])

In [142]:
obs=list(test_df['Close'])
start_prob = np.ones(len(test_states)) / len(test_states)  # or a real prior if available
Hmm = HiddenMarkovModel(trans_prob, emit_prob, prior=start_prob)
forward_algorithm(Hmm,obs)

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
N = len(test_states)
start_prob = np.ones(N) / N 

In [ ]:
backward_matrix, sequence_prob = backward_algorithm(
    obs, test_states, start_prob, trans_prob, emit_prob
)

backward_matrix

In [ ]:

print("Backward probability matrix shape:", backward_matrix.shape)
print("Probability of observation sequence under the HMM:", sequence_prob)

In [ ]:
N = len(state_labels)
start_prob = np.ones(N) / N

best_path, delta, psi = viterbi_algorithm(
    obs=obs,
    states=state_labels,
    start_prob=start_prob,
    trans_prob=trans_prob,
    emit_prob=emit_prob
)

# Map 0 to "below median ", 1 to "above median"
labels = ["below median" if state == 0 else "above median" for state in best_path]


# If you want to add this to your DataFrame
final = test_df.copy()  # avoid modifying original
final['path'] = labels

print(final[['High', 'Close', 'path']].to_string(index=False))

In [ ]:
# Test results 
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# True hidden states (e.g., High column)
true_states = test_df['High'].values

# Predicted hidden states from Viterbi
predicted_states = best_path

# Basic metrics
accuracy = accuracy_score(true_states, predicted_states)
conf_matrix = confusion_matrix(true_states, predicted_states)
precision = precision_score(true_states, predicted_states, zero_division=0)
recall = recall_score(true_states, predicted_states, zero_division=0)
f1 = f1_score(true_states, predicted_states, zero_division=0)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")